In [ ]:
import ee
import geemap
import logging
from filecmp import dircmp
import os
import filecmp
from dotenv import load_dotenv
from intake import open_catalog
import matplotlib as plt
import pandas as pd
import numpy as np
from shapely import wkt
import shapely
from sqlalchemy import create_engine, text
from shapely.geometry import shape
import geopandas as gpd
from dotenv import load_dotenv
load_dotenv()
import ipywidgets as widgets
from datetime import datetime, date, timedelta
from ipywidgets import HBox, VBox
import uuid
from datetime import datetime
import math
from ipywidgets import Button, Layout
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import asyncio

In [ ]:
auth_file = '/home/oriane/test_app/gee_credentials.json'
os.environ['EARTHENGINE_TOKEN'] = auth_file

In [ ]:
project_id = os.environ.get('GOOGLE_CLOUD_PROJECT')

ee.Authenticate()
ee.Initialize(project=project_id)

In [ ]:
### Variables
table_source = "sentinel_surfaces_detectees"
table_faits = "faits_zae_sentinel_surfaces_detectees"
table_information = "sentinel_informations_surfaces_detectees"
filter_list=['ZAE','Autre','CQ Brute','CQ PI']
mode_list=["Choix",'PI par groupe', 'CQ', 'PI FLAG','PI Entrainement']
day_collection_init=60

In [ ]:
def create_random_sampling(gdf):
    #calcul de la population à échantillonner 

    N = len(gdf) 

    e = 3   #Marge d'erreur %
    confidence_level = 0.95
    Z = 1.96 
    sample_size = (Z**2 * 0.25) / (e**2 / (N - 1))  
    sample_size = math.ceil(sample_size)

    print(f"La taille de l'échantillon est d'au moins {sample_size} pour une marge d'erreur de {e}% et un niveau de confiance de {confidence_level * 100}%.")
    print("% de la population totale = ",sample_size/N*100)
    pur_strat=sample_size/N

    sampled_df = pd.DataFrame()
    strata = gdf['her'].unique()

    for stratum in strata:
        stratum_df = gdf[gdf['her'] == stratum] 
        sample_size = int(pur_strat * len(stratum_df)) 
        stratum_sample = stratum_df.sample(n=sample_size, random_state=42)  
        sampled_df = pd.concat([sampled_df, stratum_sample]) 

    return(sampled_df)

In [ ]:
def read_data_sentinel_mode(data_type,filter):
    global date_start, date_end, table_source, table_faits, table_information

    catalog_path = f'{os.getenv("DATA_CATALOG_DIR")}/data_reference_feux.yaml'
    if data_type ==mode[1]:
        sql = f"""SELECT row_number() OVER () AS id,
        si.date_,
        si.surface_id_h3,
        ss.id_spatial AS her,
        ss.zone,
        ss.classe,
        ss.name,
        si.geometry
        FROM feux_cq.{table_source} si
        LEFT JOIN feux_cq.{table_faits} ss ON ss.surface_id_h3 = si.surface_id_h3
        WHERE ss.zone {filter} 
        AND si.date_ >= '{pd.to_datetime(date_start.value).strftime('%Y-%m-%d')}' 
        AND si.date_ <= '{pd.to_datetime(date_end.value).strftime('%Y-%m-%d')}'
        AND NOT EXISTS (
        SELECT 1 FROM feux_cq.{table_information} ti
        WHERE ti.surface_id_h3 = si.surface_id_h3)
        """
    elif data_type ==filter_list[3]:
        sql = f"""SELECT row_number() OVER () AS id,
        si.date_,
        si.surface_id_h3,
        ss.her,
        ss.validation_pi,
        si.geometry
        FROM feux_cq.{table_source} si
        LEFT JOIN feux_cq.{table_information} ss ON ss.surface_id_h3 = si.surface_id_h3
        WHERE ss.validation_pi = '{filter}' AND si.date_ >= '{pd.to_datetime(date_start.value).strftime('%Y-%m-%d')}' AND si.date_ <= '{pd.to_datetime(date_end.value).strftime('%Y-%m-%d')}'
        """
    elif data_type ==filter_list[2]:
        sql = f"""SELECT row_number() OVER () AS id,
        si.date_,
        si.surface_id_h3,
        ss.id_spatial AS her,
        si.geometry
        FROM feux_cq.{table_source} si
        LEFT JOIN feux_cq.{table_faits} ss ON ss.surface_id_h3 = si.surface_id_h3
        WHERE si.date_ >= '{pd.to_datetime(date_start.value).strftime('%Y-%m-%d')}' AND si.date_ <= '{pd.to_datetime(date_end.value).strftime('%Y-%m-%d')}'
        """
    elif data_type ==mode[3]:
        sql = f"""SELECT row_number() OVER () AS id,
        si.date_,
        si.surface_id_h3,
        ss.her,
        si.geometry
        FROM feux_cq.{table_source} si
        LEFT JOIN feux_cq.{table_information} ss ON ss.surface_id_h3 = si.surface_id_h3
        WHERE ss.validation_pi = '{filter}' AND si.date_ >= '{pd.to_datetime(date_start.value).strftime('%Y-%m-%d')}' AND si.date_ <= '{pd.to_datetime(date_end.value).strftime('%Y-%m-%d')}'
        """
    catalog = open_catalog(catalog_path)
    dataCatalog = getattr(catalog, table_source)(sql_expr=sql)
    gdf = dataCatalog.read()

    if gdf.duplicated(subset=['surface_id_h3']).sum()>=1:
        gdf = gdf.drop_duplicates(subset=['surface_id_h3'])
    
    gdf['date_']=pd.to_datetime(gdf['date_'])
    gdf = gdf.to_crs(epsg=4326)
    gdf['her'] = gdf['her'].fillna('Hors HER')

    if data_type == filter_list[3] or data_type == filter_list[2]:
        gdf = create_random_sampling(gdf)
    else:
        pass

    map_PI(gdf)

In [ ]:
class ProcessingMetadata:
    """
    Represents the metadata for a processing operation.

    Attributes:
        surface_id_h3 (str): surface h3 identification of the polygon.
        date_ (str): Date of detection of the polygon.
        her (str): Her.
        validation_pi (str): Pi validation done during normal pi.
        user_pi (int): The pi's user.
        mode_pi (int): The mode use for the pi.
        filter_pi (int): The filter use for the pi.
        date_pi (str): The date of pi.
        id_pi (str): The pi identification.

    """

    def __init__(
            self,
            surface_id_h3=None,
            date_=None,
            her=None,
            validation_pi=None,
            user_pi=None,
            mode_pi=None,
            filter_pi=None,
            id_pi=None,

            ):
        self._surface_id_h3 = surface_id_h3
        self._date_ = date_
        self._her = her
        self._validation_pi = validation_pi
        self._user_pi = user_pi
        self._mode_pi = mode_pi
        self._filter_pi = filter_pi
        self._date_pi = datetime.now()
        self._id_pi = id_pi

    def insert_metadata(self):
        metadata = {
            'surface_id_h3': self.surface_id_h3,
            'date_': self.date_,
            'her': self.her,
            'validation_pi': self.validation_pi,
            'user_pi': self.user_pi,
            'mode_pi': self.mode_pi,
            'filter_pi': self.filter_pi,
            'date_pi': datetime.now(),
            'id_pi': self.id_pi,

        }
        # Créer un DataFrame à partir du dictionnaire
        df = pd.DataFrame([metadata])
        conex = create_engine(f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PWD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{os.getenv("DB_WORKSPACE")}')

        # Insérer le DataFrame dans la base de données
        df.to_sql(table_information, schema='feux_cq', con=conex, if_exists='append', index=False)

In [ ]:
def map_PI(gdf_her_select):
    global user_widget

    run_id = str(uuid.uuid4()) ## numero unique du run de la PI
    run_id_label = widgets.Label(value='Run_id: ')
    run_id_label.value =f'Run_id: {run_id}'

    global image_viewer
    map = geemap.Map(center=(-21.2, 166), zoom=8, layout={'height': '700px', 'width': '50%'}) ### timelaps map

    left_map = map
    right_map1 = geemap.Map(center=(-21.2, 166), zoom=8, layout={'height': '700px', 'width': '25%'}) ### true color map
    right_map2 = geemap.Map(center=(-21.2, 166), zoom=8, layout={'height': '700px', 'width': '25%'}) ### false color map

    ## Widget resultat de la PI
    polygon_info = widgets.Output(layout=Layout(width='700px', height='200px', font_size='45px'))
    validate_button = widgets.Button(description="Brûlée", button_style='success',layout=Layout(width='20%', height='50px'))
    reject_button = widgets.Button(description="Non Brûlée", button_style='danger',layout=Layout(width='20%', height='50px'))
    pending_button = widgets.Button(description="FLAG", button_style='warning',layout=Layout(width='20%', height='50px'))
    choix_interval=widgets.FloatText(value=day_collection_init,description='Interval:',disabled=False)

    image_viewer = widgets.Play(interval=5000, value=0, min=0, max=1, step=1, description="Images", disabled=True) ### image viewer widget

    # Widget pour suivre les dates des images
    date_label = widgets.Label(value='Date: ')

    # Fonction pour mettre à jour les informations sur le polygone sélectionné
    def update_polygon_info(selected_gdf, index):
        with polygon_info:
            polygon_info.clear_output()
            print(f"Informations du polygone {index}:")
            print(selected_gdf.iloc[index].drop('geometry'))

    # Fonction pour mettre à jour le statut du polygone selon le choix du photo-interprète
    def update_polygon_status(new_status, utilisateur):
        global mode, filter
        
        selected_her = her_dropdown.value
        filtered_gdf = gdf_her_select[gdf_her_select['her'] == selected_her]
        current_index = polygon_selector.value
        if current_index >= 0 and current_index < len(filtered_gdf):
            filtered_index = filtered_gdf.index[current_index]
            filtered_gdf.at[filtered_index, 'Validation'] = new_status  
            filtered_gdf.at[filtered_index, 'Utilisateur'] = user_widget.value
            
            metadata = ProcessingMetadata(id_pi=run_id)
            metadata.surface_id_h3 = filtered_gdf.at[filtered_index, 'surface_id_h3']
            metadata.date_ = filtered_gdf.at[filtered_index, 'date_']
            metadata.her = selected_her
            metadata.validation_pi = new_status
            metadata.user_pi = utilisateur
            metadata.mode_pi = mode
            metadata.filter_pi = filter
            metadata.id_pi = run_id

            metadata.insert_metadata()
            
            update_polygon_info(filtered_gdf, current_index)

        else:
            with polygon_info:
                polygon_info.clear_output()
                print("L'index du polygone est invalide.")

    # Gestion des clics sur les boutons de validation
    validate_button.on_click(lambda b: update_polygon_status('Brûlée', user_widget.value))
    reject_button.on_click(lambda b: update_polygon_status('Non Brûlée', user_widget.value))
    pending_button.on_click(lambda b: update_polygon_status('FLAG', user_widget.value))

    # Fonction pour afficher les images du polygone sélectionné
    def display_images(selected_gdf, index):
        if index < len(selected_gdf):
            
            date = selected_gdf.iloc[index]['date_'] - timedelta(days=choix_interval.value)
            end_date = selected_gdf.iloc[index]['date_'] + timedelta(days=choix_interval.value)
            geom = selected_gdf.iloc[index]['geometry']
            geom_json = geom.__geo_interface__
            ee_geom = ee.Geometry(geom_json)
            buffered_geom = ee_geom.buffer(500)
            asset = ee.FeatureCollection(ee_geom)
            
            image_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filterBounds(buffered_geom.bounds()) \
                .filterDate(date, end_date)\
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70))
            
            image_fix= image_collection.filterDate(selected_gdf.iloc[index]['date_'], selected_gdf.iloc[index]['date_'] + timedelta(hours=24)).first()
            image_fix = image_fix.clip(buffered_geom.bounds())

            vis_params_polygon = {
                'color': '000000', 
                'pointSize': 3,
                'pointShape': 'circle',
                'width': 1.5,
                'lineType': 'solid',
                'fillColor': '00000000',
            }
            rgb_params={
                'bands': ['B4', 'B3', 'B2'],
                'min': 0,
                'max': 3000,
                'gamma': 1.4
            }
            fcir_params={
                'bands': ['B8', 'B4', 'B3'],
                'min': 0,
                'max': 3000,
                'gamma': 1.4
            }

            right_map1.addLayer(image_fix, rgb_params, 'RGB')
            right_map1.addLayer(asset.style(**vis_params_polygon), {}, 'Polygon')
            right_map1.centerObject(ee_geom, 17)

            right_map2.addLayer(image_fix, fcir_params, 'FCIR')
            right_map2.addLayer(asset.style(**vis_params_polygon), {}, 'Polygon')
            right_map2.centerObject(ee_geom, 17)

            # Liste des dates de la collection d'image
            dates_list = image_collection.aggregate_array('system:time_start').getInfo()
            dates_list = [datetime.utcfromtimestamp(date / 1000).strftime('%Y-%m-%d') for date in dates_list]

            slider.max=len(dates_list) - 1

            # Mettre à jour le widget PLay d'affichage des images
            image_viewer.max = len(dates_list) - 1
            image_viewer.value = 0
            image_viewer.min = 0
            image_viewer.disabled = (len(dates_list) == 0)
            image_viewer.unobserve_all()

            # Ajouter les images à la carte
            def start_image(index):
                image = ee.Image(image_collection.toList(image_collection.size()).get(index))
                image = image.clip(buffered_geom.bounds())
                
                left_map.addLayer(image, rgb_params, 'RGB')
                left_map.addLayer(asset.style(**vis_params_polygon), {}, 'Polygon')
                left_map.centerObject(ee_geom, 17) 
                             
                date_label.value = f'Date Image : {dates_list[index]}'
            
            update_polygon_info(selected_gdf, index)  
            
            widgets.jslink((image_viewer, 'value'), (slider, 'value'))
            widgets.interact(start_image, index=image_viewer)
        else:
            print("Tous les polygones ont été affichés.")

    def date_interval_change():
        current_index = polygon_selector.value
        selected_gdf = gdf_her_select[gdf_her_select['her'] == her_dropdown.value]
        display_images(selected_gdf, current_index)

    choix_interval.observe(lambda change: date_interval_change(), names='value')

    # Widget pour passer d'un polygone a un autre 
    def next_polygon(b):
        selected_her = her_dropdown.value
        filtered_gdf = gdf_her_select[gdf_her_select['her'] == selected_her]
        current_index = polygon_selector.value
        if current_index < len(filtered_gdf) - 1:
            polygon_selector.value += 1
        else:
            polygon_selector.value = 0

        display_images(filtered_gdf, polygon_selector.value)

    def previous_polygon(b):
        selected_her = her_dropdown.value
        filtered_gdf = gdf_her_select[gdf_her_select['her'] == selected_her]
        current_index = polygon_selector.value
        if current_index > 0:
            polygon_selector.value -= 1
        else:
            polygon_selector.value = len(filtered_gdf) - 1

        display_images(filtered_gdf, polygon_selector.value)
        
    next_button = widgets.Button(description="Polygone suivant", button_style='info')
    previous_button = widgets.Button(description="Polygone précédent", button_style='info')

    # widget de polygones
    polygon_selector = widgets.IntSlider(description='Avancement:',min=0, max=len(gdf_her_select) - 1, step=1, value=0)
    slider = widgets.IntSlider(step=1, value=0)

    # Dropdown pour sélectionner le HER souhaité
    her_values = sorted(set(gdf_her_select['her'].tolist()))
    her_dropdown = widgets.Dropdown(
        options=her_values,  
        description='Choisir HER:',
        disabled=False,
    )

    class MapUpdater:
        def __init__(self):
            self.filtered_gdf = None
            self.current_polygon_index = -1  
        
        def update_polygon_selector(self, change):
            selected_her = her_dropdown.value
            self.filtered_gdf = gdf_her_select[gdf_her_select['her'] == selected_her]
            if not self.filtered_gdf.empty:
                polygon_selector.max = len(self.filtered_gdf) - 1
                if self.current_polygon_index < len(self.filtered_gdf):
                    polygon_selector.value = self.current_polygon_index
                else:
                    polygon_selector.value = 0  

                display_images(self.filtered_gdf, polygon_selector.value)  # Affiche l'image au début de la série temporelle

    map_updater = MapUpdater()
    her_dropdown.observe(map_updater.update_polygon_selector, names='value')

    next_button.on_click(next_polygon)
    previous_button.on_click(previous_polygon)

    ## position des maps et widgets
    left_box = widgets.VBox([map])
    right_box = widgets.VBox([right_map1, right_map2])
    
    widgets_1=widgets.HBox([run_id_label])
    slider_polygone = widgets.HBox([polygon_selector,choix_interval])
    widgets_polygone = widgets.HBox([previous_button,next_button, her_dropdown,date_label,image_viewer,slider])
    widgets_2 = widgets.HBox([validate_button,reject_button,pending_button ])
    widgets_info=widgets.HBox([polygon_info])
    widgets_container = widgets.HBox([left_map,right_map1,right_map2])  # Ajout de date_label
    display(slider_polygone,widgets_polygone, widgets_1,widgets_container, widgets_2,widgets_info)

In [ ]:
# Création des widgets pour la sélection des dates et du mode
mode_widget = widgets.Dropdown(
    options=mode_list,
    description='Mode:',
    disabled=False
)

user_widget=widgets.Text(
    value='',
    description='User name:',
    disabled=False   
)

date_start = widgets.DatePicker(description='Date start', disabled=False, value=datetime(2021, 1, 1))
date_end = widgets.DatePicker(description='Date end', disabled=False, value=datetime(2021, 12, 31))
widgets_step1 = widgets.HBox([widgets.HBox([date_start, date_end]), mode_widget])

widgets_1=widgets.HBox([user_widget])
display(widgets_1,widgets_step1)

output_widget = widgets.Output()
display(output_widget)

class MapUpdater: 
    def __init__(self):
        self.mode_selected = "Choix"  # valeur initiale

    def selector_mode(self, change):
        global mode
        with output_widget:
            output_widget.clear_output(wait=True)  # Nettoie le contenu précédent
            self.mode_selected = change['new']  
            mode =self.mode_selected

            if change['new'] == mode_widget.options[1]:
                self.button_1 = widgets.Checkbox(description=filter_list[0])
                self.button_2 = widgets.Checkbox(description=filter_list[1])
                widgets_container = widgets.HBox([self.button_1, self.button_2])
                display(widgets_container)

                self.button_1.observe(self.handle_zae_change, names='value')
                self.button_2.observe(self.handle_autre_change, names='value')
                
            elif change['new'] == mode_widget.options[2]:
                self.button_1 = widgets.Checkbox(description=filter_list[2])
                self.button_2 = widgets.Checkbox(description=filter_list[3])
                widgets_container = widgets.HBox([self.button_1, self.button_2])
                display(widgets_container)

                self.button_1.observe(self.handle_data_CQ_BRUTE_change, names='value')
                self.button_2.observe(self.handle_data_CQ_PI_change, names='value')

            elif change['new'] == mode_widget.options[3]:
                with output_widget:
                    output_widget.clear_output(wait=True)
                    read_data_sentinel_mode(mode[3], "FLAG")


    def handle_zae_change(self, change):
        global filter
        with output_widget:
            output_widget.clear_output(wait=True)
            zae_checked = change['new']
            checkbox = change['owner']
            filter=checkbox.description
            read_data_sentinel_mode(mode[1],"IS NOT NULL")

    def handle_autre_change(self, change):
        global filter
        with output_widget:
            output_widget.clear_output(wait=True)
            autre_checked = change['new']
            checkbox = change['owner']
            filter=checkbox.description
            read_data_sentinel_mode(mode[1],"IS NULL")

    def handle_data_CQ_BRUTE_change(self, change):
        global filter
        with output_widget:
            output_widget.clear_output(wait=True)
            cq_brute_checked = change['new']
            checkbox = change['owner']
            filter=checkbox.description
            read_data_sentinel_mode(filter,None)
    
    def handle_data_CQ_PI_change(self, change):
        global filter
        with output_widget:
            output_widget.clear_output(wait=True)
            cq_pi_checked = change['new']
            checkbox = change['owner']
            filter=checkbox.description
            read_data_sentinel_mode(filter,"Brûlée")

map_updater = MapUpdater()

mode_widget.observe(map_updater.selector_mode, names='value')